In [4]:
!pip install xgboost==0.90

In [5]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import plot_importance, plot_tree

DATA_PATH = "../input/ashrae-energy-prediction/"
LEAKDATA_PATH = "../input/leakdata/"



In [6]:
train = pd.read_pickle("train.pkl")
test = pd.read_pickle("test.pkl")

In [7]:
remove_features = ['year','timestamp','meter_reading', "precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed"]
features = []
for col in train.columns:
    if col not in remove_features:
        features.append(col)
#for col in ['building_id', 'site_id','meter','primary_use','square_feet','year_built','floor_count']:
#    try:features.remove(col)
#    except:None

In [8]:
categorical_feats = []

## Train

In [9]:
num_folds = 3
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for site_id in train['site_id'].unique():
    a += 1
    print(a)
    for meter in train[train['site_id'] == site_id]['meter'].unique():
        train_small = train[(train['site_id'] == site_id) & (train['meter'] == meter)]
        test_small = test[(test['site_id'] == site_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = xgb.XGBRegressor(
                      n_estimators=6000,
                      max_depth=12,
                      num_boost_round=500,
                      learning_rate=0.05,
                      subsample=0.8,
                      colsample_bytree=0.4,
                      missing=np.nan,
                      objective ='reg:squarederror',
                      tree_method='gpu_hist'
                      )
            h = clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            %time predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading'] = predictions / num_folds
        test_small['meter_reading'] = test_small['meter_reading'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

1
[0]	validation_0-rmse:4.06736
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[203]	validation_0-rmse:0.365532

CPU times: user 425 ms, sys: 642 ms, total: 1.07 s
Wall time: 1.07 s
[0]	validation_0-rmse:3.98497
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[173]	validation_0-rmse:0.452719

CPU times: user 543 ms, sys: 556 ms, total: 1.1 s
Wall time: 1.1 s
[0]	validation_0-rmse:3.95473
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[488]	validation_0-rmse:0.634414

CPU times: user 530 ms, sys: 624 ms, total: 1.15 s
Wall time: 1.15 s
[0]	validation_0-rmse:3.77282
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[145]	validation_0-rmse:1.55069

CPU times: user 101 ms, sys: 48.2 ms, total: 149 ms
Wall time: 149 ms
[0]	validation_0-rmse:2.5885
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopp

CPU times: user 163 ms, sys: 12.5 ms, total: 175 ms
Wall time: 175 ms
[0]	validation_0-rmse:3.0449
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[221]	validation_0-rmse:1.21287

CPU times: user 122 ms, sys: 23.7 ms, total: 146 ms
Wall time: 146 ms
[0]	validation_0-rmse:4.9406
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[198]	validation_0-rmse:1.57704

CPU times: user 108 ms, sys: 39.9 ms, total: 148 ms
Wall time: 148 ms
[0]	validation_0-rmse:6.60302
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[355]	validation_0-rmse:0.176676

CPU times: user 136 ms, sys: 112 ms, total: 249 ms
Wall time: 248 ms
[0]	validation_0-rmse:6.4859
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[255]	validation_0-rmse:0.386234

CPU times: user 104 ms, sys: 128 ms, total: 232 ms
Wall time: 232 ms
[0]	validation_0-rmse:6.51083
Wil

Stopping. Best iteration:
[84]	validation_0-rmse:1.56402

CPU times: user 33.6 ms, sys: 0 ns, total: 33.6 ms
Wall time: 33.6 ms
[0]	validation_0-rmse:2.31453
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[35]	validation_0-rmse:1.68001

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 32.1 ms
12
[0]	validation_0-rmse:3.5349
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[167]	validation_0-rmse:0.331092

CPU times: user 326 ms, sys: 436 ms, total: 763 ms
Wall time: 762 ms
[0]	validation_0-rmse:3.46696
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[301]	validation_0-rmse:0.304855

CPU times: user 324 ms, sys: 464 ms, total: 788 ms
Wall time: 788 ms
[0]	validation_0-rmse:3.52158
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[585]	validation_0-rmse:0.324337

CPU times: user 449 ms, sys: 476 ms, total: 

[0]	validation_0-rmse:5.49339
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[161]	validation_0-rmse:0.763453

CPU times: user 614 ms, sys: 780 ms, total: 1.39 s
Wall time: 1.39 s
16
[0]	validation_0-rmse:6.14436
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[419]	validation_0-rmse:1.53426

CPU times: user 253 ms, sys: 284 ms, total: 537 ms
Wall time: 536 ms
[0]	validation_0-rmse:6.60968
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[136]	validation_0-rmse:1.9029

CPU times: user 205 ms, sys: 284 ms, total: 489 ms
Wall time: 489 ms
[0]	validation_0-rmse:6.12946
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[371]	validation_0-rmse:1.65014

CPU times: user 239 ms, sys: 296 ms, total: 535 ms
Wall time: 535 ms
[0]	validation_0-rmse:4.67824
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopp

In [10]:
num_folds = 4
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for building_id in train['building_id'].unique():
    a += 1
    print(a)
    for meter in train[train['building_id'] == building_id]['meter'].unique():
        train_small = train[(train['building_id'] == building_id) & (train['meter'] == meter)]
        test_small = test[(test['building_id'] == building_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = xgb.XGBRegressor(
                      n_estimators=6000,
                      max_depth=12,
                      num_boost_round=500,
                      learning_rate=0.1,
                      subsample=0.8,
                      colsample_bytree=0.4,
                      missing=np.nan,
                      objective ='reg:squarederror',
                      tree_method='gpu_hist'
                      )
            h = clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=50,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            %time predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_building_id'] = predictions / num_folds
        test_small['meter_reading_building_id'] = test_small['meter_reading_building_id'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_building_id']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_building_id']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

1
[0]	validation_0-rmse:3.45094
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[169]	validation_0-rmse:0.237373

CPU times: user 12.8 ms, sys: 0 ns, total: 12.8 ms
Wall time: 12.8 ms
[0]	validation_0-rmse:3.43884
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[169]	validation_0-rmse:0.176213

CPU times: user 13.5 ms, sys: 227 µs, total: 13.8 ms
Wall time: 13.8 ms
[0]	validation_0-rmse:3.44421
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[46]	validation_0-rmse:0.168096

CPU times: user 10.9 ms, sys: 3.89 ms, total: 14.8 ms
Wall time: 14.8 ms
[0]	validation_0-rmse:3.45932
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[212]	validation_0-rmse:0.259555

CPU times: user 16.5 ms, sys: 118 µs, total: 16.6 ms
Wall time: 16.6 ms
2
[0]	validation_0-rmse:1.82756
Will train until validation_0-rmse hasn't improved in 50 rou

Stopping. Best iteration:
[84]	validation_0-rmse:1.8906

CPU times: user 11.9 ms, sys: 3.94 ms, total: 15.8 ms
Wall time: 15.8 ms
[0]	validation_0-rmse:3.23285
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[57]	validation_0-rmse:1.874

CPU times: user 14.4 ms, sys: 360 µs, total: 14.8 ms
Wall time: 14.8 ms
[0]	validation_0-rmse:0.59345
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[0]	validation_0-rmse:0.59345

CPU times: user 4.7 ms, sys: 8.1 ms, total: 12.8 ms
Wall time: 12.8 ms
[0]	validation_0-rmse:2.18799
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[18]	validation_0-rmse:1.98633

CPU times: user 14.9 ms, sys: 0 ns, total: 14.9 ms
Wall time: 14.9 ms
[0]	validation_0-rmse:3.67876
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[176]	validation_0-rmse:0.113918

CPU times: user 20.3 ms, sys: 56 µs, total: 20

Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[99]	validation_0-rmse:0.788977

CPU times: user 13.5 ms, sys: 4.26 ms, total: 17.7 ms
Wall time: 17.7 ms
[0]	validation_0-rmse:2.05414
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[17]	validation_0-rmse:1.10847

CPU times: user 6.09 ms, sys: 7.75 ms, total: 13.8 ms
Wall time: 13.8 ms
[0]	validation_0-rmse:2.9306
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[55]	validation_0-rmse:1.89635

CPU times: user 14.9 ms, sys: 0 ns, total: 14.9 ms
Wall time: 14.9 ms
14
[0]	validation_0-rmse:4.85574
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[300]	validation_0-rmse:0.303573

CPU times: user 17.2 ms, sys: 225 µs, total: 17.5 ms
Wall time: 17.5 ms
[0]	validation_0-rmse:4.75474
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[376

Stopping. Best iteration:
[201]	validation_0-rmse:0.314351

CPU times: user 13.8 ms, sys: 3.69 ms, total: 17.5 ms
Wall time: 17.5 ms
[0]	validation_0-rmse:2.66583
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[25]	validation_0-rmse:0.57797

CPU times: user 17 ms, sys: 189 µs, total: 17.2 ms
Wall time: 17.2 ms
[0]	validation_0-rmse:2.64359
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[307]	validation_0-rmse:0.387737

CPU times: user 16.1 ms, sys: 57 µs, total: 16.2 ms
Wall time: 16.2 ms
21
[0]	validation_0-rmse:4.0907
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[212]	validation_0-rmse:0.071298

CPU times: user 13.2 ms, sys: 4.14 ms, total: 17.4 ms
Wall time: 17.3 ms
[0]	validation_0-rmse:4.08118
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[47]	validation_0-rmse:0.057046

CPU times: user 9.5 ms, sys: 4.29 

CPU times: user 9.5 ms, sys: 7.65 ms, total: 17.1 ms
Wall time: 17.2 ms
[0]	validation_0-rmse:2.88592
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[31]	validation_0-rmse:0.126432

CPU times: user 15.9 ms, sys: 0 ns, total: 15.9 ms
Wall time: 15.9 ms
[0]	validation_0-rmse:3.2575
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[218]	validation_0-rmse:0.449605

CPU times: user 16 ms, sys: 43 µs, total: 16.1 ms
Wall time: 16.1 ms
30
[0]	validation_0-rmse:3.55614
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[260]	validation_0-rmse:0.249775

CPU times: user 8.84 ms, sys: 8.09 ms, total: 16.9 ms
Wall time: 16.9 ms
[0]	validation_0-rmse:3.09967
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[74]	validation_0-rmse:0.176738

CPU times: user 14.8 ms, sys: 0 ns, total: 14.8 ms
Wall time: 14.8 ms
[0]	validation_0-rmse:2.76

...

CPU times: user 13.2 ms, sys: 1.33 ms, total: 14.5 ms
Wall time: 14.5 ms
[0]	validation_0-rmse:6.13059
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[129]	validation_0-rmse:0.092341

CPU times: user 10.3 ms, sys: 5.44 ms, total: 15.8 ms
Wall time: 15.8 ms
[0]	validation_0-rmse:6.07269
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[44]	validation_0-rmse:0.128701

CPU times: user 6.73 ms, sys: 6.1 ms, total: 12.8 ms
Wall time: 12.8 ms
1421
[0]	validation_0-rmse:4.69738
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[124]	validation_0-rmse:0.114799

CPU times: user 10.6 ms, sys: 2.08 ms, total: 12.6 ms
Wall time: 12.6 ms
[0]	validation_0-rmse:4.72029
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[37]	validation_0-rmse:0.137729

CPU times: user 12.8 ms, sys: 0 ns, total: 12.8 ms
Wall time: 12.8 ms
[0]	validation_0

CPU times: user 14 ms, sys: 0 ns, total: 14 ms
Wall time: 14 ms
[0]	validation_0-rmse:3.12774
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[100]	validation_0-rmse:0.112396

CPU times: user 9.68 ms, sys: 976 µs, total: 10.7 ms
Wall time: 10.7 ms
[0]	validation_0-rmse:3.16586
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[101]	validation_0-rmse:0.222455

CPU times: user 10.5 ms, sys: 1.06 ms, total: 11.6 ms
Wall time: 11.6 ms
1430
[0]	validation_0-rmse:3.42424
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[33]	validation_0-rmse:0.115599

CPU times: user 4.56 ms, sys: 9.27 ms, total: 13.8 ms
Wall time: 13.8 ms
[0]	validation_0-rmse:3.53637
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[178]	validation_0-rmse:0.137121

CPU times: user 10.6 ms, sys: 1.07 ms, total: 11.6 ms
Wall time: 11.6 ms
[0]	validation_0-rmse

Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[464]	validation_0-rmse:0.167638

CPU times: user 12.6 ms, sys: 2.3 ms, total: 14.9 ms
Wall time: 15 ms
[0]	validation_0-rmse:0.667184
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[261]	validation_0-rmse:0.085945

CPU times: user 13.7 ms, sys: 0 ns, total: 13.7 ms
Wall time: 13.7 ms
1435
[0]	validation_0-rmse:2.12694
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[107]	validation_0-rmse:0.092587

CPU times: user 7.47 ms, sys: 5.16 ms, total: 12.6 ms
Wall time: 12.6 ms
[0]	validation_0-rmse:2.04737
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[97]	validation_0-rmse:0.138312

CPU times: user 12.4 ms, sys: 2.27 ms, total: 14.7 ms
Wall time: 14.7 ms
[0]	validation_0-rmse:1.77254
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration

Stopping. Best iteration:
[90]	validation_0-rmse:0.101715

CPU times: user 13.4 ms, sys: 1.35 ms, total: 14.7 ms
Wall time: 14.8 ms
[0]	validation_0-rmse:2.11567
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[108]	validation_0-rmse:0.110299

CPU times: user 6.54 ms, sys: 9.47 ms, total: 16 ms
Wall time: 16 ms
1441
[0]	validation_0-rmse:2.29664
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[165]	validation_0-rmse:0.362613

CPU times: user 14.8 ms, sys: 0 ns, total: 14.8 ms
Wall time: 14.8 ms
[0]	validation_0-rmse:2.25734
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[111]	validation_0-rmse:0.230186

CPU times: user 8.23 ms, sys: 5.23 ms, total: 13.5 ms
Wall time: 13.5 ms
[0]	validation_0-rmse:2.1459
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[309]	validation_0-rmse:0.205414

CPU times: user 12.9 ms, sys: 1.

CPU times: user 10.8 ms, sys: 2.11 ms, total: 12.9 ms
Wall time: 12.9 ms
[0]	validation_0-rmse:5.30262
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[49]	validation_0-rmse:0.024949

CPU times: user 6.41 ms, sys: 6.07 ms, total: 12.5 ms
Wall time: 12.5 ms
[0]	validation_0-rmse:6.58461
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[63]	validation_0-rmse:0.141056

CPU times: user 12 ms, sys: 1.21 ms, total: 13.2 ms
Wall time: 13.2 ms
[0]	validation_0-rmse:6.62283
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[170]	validation_0-rmse:0.101288

CPU times: user 12.4 ms, sys: 1.25 ms, total: 13.6 ms
Wall time: 13.6 ms
[0]	validation_0-rmse:6.63998
Will train until validation_0-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[124]	validation_0-rmse:0.098078

CPU times: user 11 ms, sys: 2.12 ms, total: 13.1 ms
Wall time: 13.1 ms
[0]	validation_0-rmse

In [11]:
num_folds = 4
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for primary_use in train['primary_use'].unique():
    a += 1
    print(a, primary_use)
    for meter in train[train['primary_use'] == primary_use]['meter'].unique():
        train_small = train[(train['primary_use'] == primary_use) & (train['meter'] == meter)]
        test_small = test[(test['primary_use'] == primary_use) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

            clf = xgb.XGBRegressor(
                      n_estimators=6000,
                      max_depth=12,
                      num_boost_round=500,
                      learning_rate=0.05,
                      subsample=0.8,
                      colsample_bytree=0.4,
                      missing=np.nan,
                      objective ='reg:squarederror',
                      tree_method='gpu_hist'
                      )
            #h = clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=150,verbose=2000)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            %time predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_primary_use'] = predictions / num_folds
        test_small['meter_reading_primary_use'] = test_small['meter_reading_primary_use'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_primary_use']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_primary_use']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

1 0
[0]	validation_0-rmse:4.04936
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[137]	validation_0-rmse:0.354885

CPU times: user 4.66 s, sys: 6.27 s, total: 10.9 s
Wall time: 10.9 s
[0]	validation_0-rmse:4.05736
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[586]	validation_0-rmse:0.291958

CPU times: user 5.78 s, sys: 6.61 s, total: 12.4 s
Wall time: 12.4 s
[0]	validation_0-rmse:4.15624
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[258]	validation_0-rmse:0.302826

CPU times: user 4.93 s, sys: 6.35 s, total: 11.3 s
Wall time: 11.3 s
[0]	validation_0-rmse:4.085
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[137]	validation_0-rmse:0.410995

CPU times: user 4.69 s, sys: 6.09 s, total: 10.8 s
Wall time: 10.8 s
[0]	validation_0-rmse:4.15569
Will train until validation_0-rmse hasn't improved in 150 rounds.
St

CPU times: user 349 ms, sys: 537 ms, total: 887 ms
Wall time: 887 ms
[0]	validation_0-rmse:4.40733
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[142]	validation_0-rmse:1.08037

CPU times: user 385 ms, sys: 524 ms, total: 909 ms
Wall time: 909 ms
[0]	validation_0-rmse:5.06681
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[123]	validation_0-rmse:1.5146

CPU times: user 412 ms, sys: 487 ms, total: 899 ms
Wall time: 899 ms
[0]	validation_0-rmse:3.84961
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[50]	validation_0-rmse:1.46378

CPU times: user 350 ms, sys: 537 ms, total: 887 ms
Wall time: 887 ms
[0]	validation_0-rmse:4.7288
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[100]	validation_0-rmse:2.28589

CPU times: user 138 ms, sys: 32.2 ms, total: 170 ms
Wall time: 170 ms
[0]	validation_0-rmse:3.61301
Will tr

Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[292]	validation_0-rmse:1.00749

CPU times: user 1.3 s, sys: 1.63 s, total: 2.93 s
Wall time: 2.93 s
[0]	validation_0-rmse:4.59219
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[884]	validation_0-rmse:0.977136

CPU times: user 1.72 s, sys: 1.74 s, total: 3.46 s
Wall time: 3.46 s
[0]	validation_0-rmse:3.27522
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[58]	validation_0-rmse:1.25617

CPU times: user 1.17 s, sys: 1.57 s, total: 2.74 s
Wall time: 2.74 s
[0]	validation_0-rmse:5.94001
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[199]	validation_0-rmse:1.10228

CPU times: user 669 ms, sys: 972 ms, total: 1.64 s
Wall time: 1.64 s
[0]	validation_0-rmse:5.01071
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[167]	validat

Stopping. Best iteration:
[72]	validation_0-rmse:0.820051

CPU times: user 16.8 ms, sys: 1.77 ms, total: 18.6 ms
Wall time: 18.6 ms
[0]	validation_0-rmse:4.73612
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[64]	validation_0-rmse:0.772185

CPU times: user 19 ms, sys: 0 ns, total: 19 ms
Wall time: 19 ms
[0]	validation_0-rmse:6.14755
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[165]	validation_0-rmse:1.16603

CPU times: user 21.2 ms, sys: 0 ns, total: 21.2 ms
Wall time: 21.2 ms
9 10
[0]	validation_0-rmse:1.14112
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[123]	validation_0-rmse:0.37956

CPU times: user 27.6 ms, sys: 2.9 ms, total: 30.5 ms
Wall time: 30.5 ms
[0]	validation_0-rmse:1.22159
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[187]	validation_0-rmse:0.31817

CPU times: user 31.4 ms, sys: 0 ns, t

Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[161]	validation_0-rmse:0.634693

CPU times: user 29.7 ms, sys: 0 ns, total: 29.7 ms
Wall time: 29.7 ms
[0]	validation_0-rmse:3.40146
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[17]	validation_0-rmse:2.26977

CPU times: user 25.2 ms, sys: 0 ns, total: 25.2 ms
Wall time: 25.2 ms
12 14
[0]	validation_0-rmse:3.86061
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[263]	validation_0-rmse:0.346163

CPU times: user 42.3 ms, sys: 914 µs, total: 43.2 ms
Wall time: 43.2 ms
[0]	validation_0-rmse:3.88813
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[1037]	validation_0-rmse:0.313762

CPU times: user 59.7 ms, sys: 13.4 ms, total: 73.1 ms
Wall time: 73.1 ms
[0]	validation_0-rmse:3.96779
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best itera

Stopping. Best iteration:
[5]	validation_0-rmse:0.69808

CPU times: user 9.1 ms, sys: 956 µs, total: 10.1 ms
Wall time: 10.1 ms
[0]	validation_0-rmse:3.79496
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[152]	validation_0-rmse:3.28108

CPU times: user 11.3 ms, sys: 1.19 ms, total: 12.5 ms
Wall time: 12.5 ms
[0]	validation_0-rmse:4.47258
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[183]	validation_0-rmse:1.04173

CPU times: user 12.2 ms, sys: 0 ns, total: 12.2 ms
Wall time: 12.3 ms
[0]	validation_0-rmse:3.64524
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[68]	validation_0-rmse:0.826699

CPU times: user 10.8 ms, sys: 0 ns, total: 10.8 ms
Wall time: 10.8 ms
[0]	validation_0-rmse:3.68316
Will train until validation_0-rmse hasn't improved in 150 rounds.
Stopping. Best iteration:
[315]	validation_0-rmse:0.456557

CPU times: user 14.5 ms, sys: 0 ns, to

In [12]:
sub = pd.read_csv(DATA_PATH + "sample_submission.csv")
sub["meter_reading"] = test['meter_reading'] * 0.6 + test['meter_reading_primary_use'] * 0.2 + test['meter_reading_building_id'] * 0.2
#sub.to_csv("best_xgb.gz",  index = False, compression= 'gzip')

In [13]:
#test = test.rename(columns={'meter_reading':'meter_reading_site_id'})
#test["meter_reading"] = test['meter_reading_site_id'] #* 0.6 + test['meter_reading_primary_use'] * 0.2 + test['meter_reading_building_id'] * 0.2

In [14]:
test['meter_reading'] = sub["meter_reading"]

In [15]:
test['meter_reading'] = [meter_r * 0.85 if site not in [0,1,2,3,4,15] else meter_r for meter_r,site in zip(test['meter_reading'], test['site_id'])]

In [16]:
leak_df = pd.read_csv(LEAKDATA_PATH+'leak.csv')
leak_df['timestamp'] = pd.to_datetime(leak_df["timestamp"])
leak_df.dropna(0, inplace=True)
leak_df = leak_df[(leak_df.timestamp.dt.year > 2016) & (leak_df.timestamp.dt.year < 2019)]
leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0 # remove large negative values
leak_df = leak_df[leak_df.building_id!=245]
leak_df['meter_reading_sc'] = leak_df['meter_reading'] 
del leak_df['meter_reading'] 
test = test.merge(leak_df, on=['building_id','meter','timestamp'], how='left')

In [17]:
test['meter_reading'] = [a if (a >= 0 and site != 4) or (a < 0 and site != 4) else b for a,b,site in zip(test['meter_reading_sc'], test['meter_reading'], test['site_id'])]

In [18]:
sub["meter_reading"] = test['meter_reading']
sub.to_csv("../sub/xgb_3wise.csv",  index = False,float_format="%.4f")